In [ ]:
# pip install python-kasa flask requests picamera

In [ ]:
'''
sudo iptables -t nat -A PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080 
sudo iptables -t nat -D PREROUTING -p tcp --dport 80 -j REDIRECT --to-ports 8080
''' 

import json # dhcp-lease-list
sec = {'kitchen_bulb_ip':"192.168.x.x", 
       'bedroom_bulb_ip':"192.168.x.x",
       'bedroom_ac_ip'  :"192.168.x.x", 
       'hub_ip'         :"192.168.x.x"} 

#with open('secret.json','w') as fw: json.dump(sec, fw)

In [ ]:
import sqlite3, datetime, time, queue, threading, sys, os, asyncio, kasa

with open('secret.json') as fr: sec = json.load(fr) 

def initialize_database(): 
    con = sqlite3.connect(f'SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    con.commit() 
    con.close()
    
#initialize_database()

In [ ]:
def database(): 
    con = sqlite3.connect('SHT40.2021-09-08T19:29:27.db')  
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [ ]:
import dateutil, requests
UTC = dateutil.tz.gettz('UTC') 
class HUB:
    def __init__(self, temp_ceil = 28.5, lumi_ceil = 650):
        self.ac              = kasa.SmartPlug(sec['bedroom_ac_ip'])   
        self.bulb            = kasa.SmartBulb(sec['bedroom_bulb_ip'])   
        self.params          = { 'temp_ceil':temp_ceil,
                                 'lumi_ceil':lumi_ceil, 
                                'night_mode':False,
                               'manual_mode':False,
                                'sunrise_dt':None,
                                'sunset_dt' :None,
                                  'relay_on':False
                               }
        threading.Thread(target=self.update).start()  
        time.sleep(3) 
            
    def update(self):
        lat, lng = (40.7063311, -73.9971931)
        while 1:
            print('[Daily Sunrise & Sunset Update]')
            date = datetime.datetime.now().strftime('%Y-%m-%d')
            r = requests.get(f'https://api.sunrise-sunset.org/json?lat={lat}&lng={lng}&date={date}&formatted=0')
            if r.json()['status'] != 'OK': raise
            self.params['sunrise_dt'] = datetime.datetime.fromisoformat(r.json()['results']['sunrise'])
            self.params['sunset_dt']  = datetime.datetime.fromisoformat(r.json()['results']['sunset']) 
            while 1: 
                if self.params['sunrise_dt'].day != datetime.datetime.now().day: 
                    print('A New Day Has Come')
                    break
                asyncio.run(self.ac.update())   
                asyncio.run(self.bulb.update())
                if datetime.datetime.now().astimezone(UTC) < self.params['sunrise_dt'] or \
                   datetime.datetime.now().astimezone(UTC) > self.params['sunset_dt']: # Daylight
                    self.params['night_mode'] = True 
                else:                                                        # At night
                    self.params['night_mode'] = False
                time.sleep(60)  

hub = HUB()

In [ ]:
index_html_string = ''' <!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title> hub </title>
    <style>
      .inp {height: 100px; width: 100px}  
      
      #slider {
        -webkit-appearance: none;
        width: 100%;
        height: 25px;
        background: #d3d3d3;
        outline: none;
        opacity: 0.7;
        -webkit-transition: .2s;
        transition: opacity .2s;
      }
      
      #slider:hover {
        opacity: 1;
      } 
      
      #slider::-webkit-slider-thumb {
        -webkit-appearance: none;
        appearance: none;
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      }
      
      #slider::-moz-range-thumb {
        width: 100px;
        height: 100px;
        background: #04AA6D;
        cursor: pointer;
      } 
      
      td {text-align: center; }
    </style>
  </head>
  <body> 
    <form method="POST" action="/" enctype="multipart/form-data"> 
      <table style="width: 500px;" > 
        <tr>
          <td class="inp">Light Mode:</td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value="1" {{'checked' if hub.params['manual_mode'] else ''}} class="inp">
            <span style="position: absolute;">Manu</span> 
          </td> 
          <td style="position: relative;">
            <input type="radio" name="manual_mode" value='0' {{'' if hub.params['manual_mode'] else 'checked'}} class="inp"> 
            <span style="position: absolute; top: 0px; left: 0px; ">Auto</span> 
          </td> 
        </tr>  
        <tr>
          <td>Max Celsius Degress:</td>
          <td><input type="text" name="temp_ceil" value="{{hub.params['temp_ceil']}}" style="font-size: 40px;" class="inp" id="output"/></td>
          <td><input type="submit" value="Submit" class="inp"></td>
        </tr>  
        <tr >
          <td colspan="3"><input type="range" min="1" max="100" value="{{ 10*hub.params['temp_ceil']-200}}" style="height:100px; width:100%" class="inp" id="slider"></td>
          <script>
            var output = document.getElementById("output");
            var slider = document.getElementById("slider");  
            slider.oninput = function() {
              output.setAttribute("value", 20+slider.value/10);   
            }
          </script> 
        </tr>
      </table>  
      <br/> 
    </form>  
  </body>
</html> 
'''

import flask, logging

log = logging.getLogger('werkzeug') 
#log.setLevel(logging.ERROR) 

app = flask.Flask(__name__)  

@app.route("/", methods=['GET', 'POST'])
def index(): 
    req = flask.request 
    if req.method == 'GET':
        return flask.render_template_string(index_html_string, hub=hub) 
    elif req.method == 'POST':  
        hub.params['temp_ceil'] = float(req.form['temp_ceil']) 
        hub.params['manual_mode'] = int(req.form['manual_mode']) 
        return flask.redirect("/", code=302)
    
@app.route("/remote", methods=['GET'])
def remote(): 
    pin = flask.request.args.get('pin', -1)
    if   pin ==  '0' : 
        if hub.params['relay_on']: 
            requests.post(f"http://{hub.params['relay_ip']}:{hub.params['relay_port']}/params",json={'switch': 0}) 
            hub.params.update({'relay_on':False})
            d = 'Relay Off'
        else: 
            requests.post(f"http://{hub.params['relay_ip']}:{hub.params['relay_port']}/params",json={'switch': 1}) 
            hub.params.update({'relay_on':True}) 
            d = 'Relay On'  
    elif pin ==  '2' :  
        d = 'Pin 2'
    elif pin ==  '4' :  
        d = 'Pin 4'
    elif pin ==  '5' :  
        d = 'Pin 5'
    elif pin == '12' :  
        hub.params['temp_ceil'] += 1
        d = f'Ceil: {hub.params["temp_ceil"]}'
    elif pin == '13' :  
        if hub.ac.is_on:
            asyncio.run(hub.ac.turn_off()) 
            asyncio.run(hub.ac.update())   
            d = 'A/C Off' 
        else:
            asyncio.run(hub.ac.turn_on()) 
            asyncio.run(hub.ac.update())  
            d = 'A/C On' 
    elif pin == '14' : 
        hub.params['temp_ceil'] -= 1
        d = f'Ceil: {hub.params["temp_ceil"]}'
    elif pin == '15' : # 
        if hub.bulb.is_on:
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())   
            hub.params['manual_mode'] = True
            d = 'LightOff'
        else:
            asyncio.run(hub.bulb.turn_on()) 
            asyncio.run(hub.bulb.update())   
            hub.params['manual_mode'] = False
            d = 'LightON'
    return flask.jsonify({'display':d})
    
@app.route("/log", methods=['POST'])
def log():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor']
    
    if T > hub.params['temp_ceil'] and hub.ac.is_on == False: 
        asyncio.run(hub.ac.turn_on()) 
        asyncio.run(hub.ac.update())  
    elif T < hub.params['temp_ceil'] - 1 and hub.ac.is_on == True:
        asyncio.run(hub.ac.turn_off()) 
        asyncio.run(hub.ac.update())  
        
    if hub.params['manual_mode']:
        pass
    else:
        if hub.params['night_mode']:
            if P < hub.params['lumi_ceil']:
                asyncio.run(hub.bulb.turn_on()) 
                asyncio.run(hub.bulb.update()) 
            elif hub.bulb.is_on: 
                asyncio.run(hub.bulb.turn_off()) 
                asyncio.run(hub.bulb.update())
        else:
            asyncio.run(hub.bulb.turn_off()) 
            asyncio.run(hub.bulb.update())
        
    database_operator(('insert', flask.request.json)) 
    return '' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }
 

@app.route("/params", methods=['GET', 'POST'])
def params():   
    req = flask.request 
    if req.method == 'GET':
        return hub.params
    elif req.method == 'POST':  #  
        hub.params.update(req.json) 
        return {'return_code':100}

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'
 
app.run(sec['hub_ip'],'8080')
# database_operator(('sql', {'sql':'select * from log'})) 

In [ ]:
database_operator(('commit', ''))

In [ ]:
hub.params

In [ ]:
#!jupyter nbconvert --to script hub.ipynb